In [21]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from spacy.cli import download
from chatterbot.trainers import ListTrainer
#Solução para corrigir um bug nas bibliotecas que ocorre com o chatterbot
download('en_core_web_sm')
class ENGSM:
    ISO_639_1 = 'en_core_web_sm'
##############################################

chatbot = ChatBot('BotAssistente', tagger_language= ENGSM)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Codigo principal 

In [28]:
import datetime
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

from fuzzywuzzy import process # Biblioteca para correlacionar pontuações entre o usuario e o chat, com ela identifcamos a pergunta e através de pontuações ele escolhe a melhor resposta 
agora = datetime.datetime.now()
hora = agora.strftime("%H:%M:%S")

chatbot = ChatBot('BotAssistente', tagger_language=ENGSM) # Iniciação do chatbot
# Lista a ser treinada
conversa = [
    "Olá, como você está?",
    "Estou bem, obrigado! E você?",
    "Bom dia, tudo bem?",
    "Bom dia, tudo sim e com você?",
    "Olá, tudo bem?",
    "Sim e aí, tudo tranquilo?",
    "Que horas são?",
    f"São {hora}",
    "Está tudo bem?",
    "Tudo certo e por aí?",
    "Oi, qual é a boa?",
    "Oi! A boa é estar aqui para ajudar. E contigo?",
    "Oi, como foi o seu dia?",
    "Oi! Meu dia foi bom, obrigado. E o seu?",
    "Salve",
    "Salve! Como posso contribuir para o seu dia?",
    "Oi, pode me tirar uma dúvida?",
    "Sim, claro! Qual é a sua dúvida?",
    "Olá",
    "Olá, tudo bem? Como posso ajudar?",
    "Oi, tudo bem?",
    "Tudo sim, como você está?",
    "Como vai o trabalho?",
    "O trabalho vai bem, obrigado. E o seu?",
    "Você tem alguma ideia de negócio?",
    "Sim, tenho várias! Se quiser, posso compartilhar algumas ideias de negócios.",
    "O que você faz da vida?",
    "Trabalho na área de tecnologia. E você?",
    "Como está o tempo hoje?",
    "Está ensolarado e agradável. Um ótimo dia para fazer algo ao ar livre!",
    "Você gosta de viajar?",
    "Sim, adoro viajar! Já conheci vários lugares incríveis.",
    "Qual é o seu hobby?",
    "Gosto de ler e aprender coisas novas. E você?",
    "Você já pensou em empreender?",
    "Sim, empreender é uma ótima maneira de transformar ideias em realidade. Tem algum projeto em mente?",
    "Qual é o seu filme favorito?",
    "Difícil escolher apenas um, mas gosto muito de filmes de ficção científica. E você?",
    "Qual é a sua comida favorita?",
    "Gosto de comida italiana. E você?",
    "Você pratica algum esporte?",
    "Sim, tento manter uma rotina de exercícios. Ajuda a manter a mente saudável!",
    "Qual é a última música que você ouviu?",
    "Ouvi uma música nova da minha playlist hoje. E você, tem alguma recomendação?",
    "O que você acha sobre inteligência artificial?",
    "A inteligência artificial é fascinante! Ela tem o potencial de transformar muitos setores.",
    "Qual é a sua opinião sobre política?",
    "Política é um assunto complexo, mas é importante se envolver e se informar para tomar decisões conscientes.",
    "Você já ouviu falar sobre a teoria das cordas?",
    "Sim, é uma teoria intrigante na física. Você tem interesse em teorias científicas?",
    "O que você acha sobre viagens espaciais?",
    "Viagens espaciais são incríveis! O avanço da exploração espacial abre novas possibilidades para o futuro.",
    "Qual é o último livro que você leu?",
    "Li um livro sobre inteligência artificial recentemente. E você, tem alguma recomendação?",
    "O que você pensa sobre energia renovável?",
    "Energia renovável é essencial para o futuro sustentável do planeta. Você tem práticas sustentáveis?",
]
with open("perguntas.txt", "w") as file: # abrindo um arquivo txt para salvar todas as perguntas do usuario pra futuramenter uma melhoria 
    file.write("Perguntas feitas pelo usuário:\n\n")

trainer = ListTrainer(chatbot)
trainer.train(conversa)

while True:
    mensagem = input('Digite aqui a mensagem: ')
    if mensagem == '':
        break
    else:
        # usando a biblioteca fuzzy pra achar a melhor correspondencia 
        melhor_correspondencia, pontuacao = process.extractOne(mensagem, conversa)
        
        with open("perguntas.txt", "a") as file: # Colocando a pergunta no arquivo txt
            file.write(f"{mensagem}\n")
        if pontuacao >= 70:  # definindo limite de acertetivade 
            print(chatbot.get_response(melhor_correspondencia))
        else:
            print('Desculpe, no momento não temos uma resposta para isso, por isso iremos salvar sua pergunta para trabalharmos nisso')

# Limpar banco de dados
chatbot.storage.drop()


List Trainer: [####################] 100%
Digite aqui a mensagem: Olá tudo bem ?
Tudo certo e por aí?
Digite aqui a mensagem: Sim como você está ?
Como vai o trabalho?


KeyboardInterrupt: Interrupted by user

## Codigo exemplo de uma outra maneira 

In [12]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from google.cloud import dialogflow
from google.oauth2 import service_account
import uuid
import os
caminho_credenciais = 'C:/Users/rapha/Documents/GitHub/chatbotemprego/chatbot-jyc9-d10cfe6dcda0.json' # Caminho de autenticações de credenciais instaladas de acordo e pedido pelo doc do dialog flow
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = caminho_credenciais
# Configuração do ChatBot
chatbot = ChatBot(
    'BotAssistente',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': 'Desculpe, não entendi. Pode reformular?',
            'maximum_similarity_threshold': 0.90
        }
    ], tagger_language= ENGSM
) # Configurando o bot 

# Criando um trainer
trainer = ChatterBotCorpusTrainer(chatbot)

# Treinando para portugues
trainer.train('chatterbot.corpus.portuguese')

# Passando IDs para o dialog flow funcionar 
project_id = "chatbot-jyc9"
language_code = "pt-BR"

def buscando_intent(text):
    session_client = dialogflow.SessionsClient()
    session_id = str(uuid.uuid4())  # Gera um UUID único para cada sessão
    session = session_client.session_path(project_id, session_id)

    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)

    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )

    return response.query_result.fulfillment_text

# Função para obter a resposta do chatbot
def obter_resposta(pergunta):

    resposta_dialogflow = busncando_intent(pergunta)

    # Se o Dialogflow não retornar uma resposta útil ele utlizao  chatter bot
    if not resposta_dialogflow:
        resposta_chatbot = str(chatbot.get_response(pergunta))
        return resposta_chatbot

    return resposta_dialogflow

while True:
    pergunta_usuario = input("Você: ")
    if pergunta_usuario.lower() == 'sair':
        break
    resposta_bot = obter_resposta(pergunta_usuario)
    print(f"ChatBot: {resposta_bot}")


Training compliment.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training linguistic_knowledge.yml: [####################] 100%
Training proverbs.yml: [####################] 100%
Training suggestions.yml: [####################] 100%
Training trivia.yml: [####################] 100%
Training unilab.yml: [####################] 100%
Você: Olá tudo bem ?
ChatBot: Olá!
Você: Como vai ?
ChatBot: Lamento, mas não compreendi.


KeyboardInterrupt: Interrupted by user